In [ ]:
import spherpro.bro as spb
import spherpro.datastore as spd
import spherpro.library as spl
import spherpro.configuration as conf
import spherpro.db as db
import sqlalchemy as sa
import imp
import pycytools as pct
import pycytools.library
import re
import os
import pandas as pd
import numpy as np
import plotnine as gg
import spherpro.library as lib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from scipy.cluster import hierarchy

import pdb
import pathlib

%matplotlib notebook
%matplotlib inline


import seaborn as sns
import scipy.stats as stats
from IPython.display import display


from pandas.api.types import CategoricalDtype

In [ ]:
sm = snakemake

# Investigate the general correlation structure




In [ ]:
import spherpro.bromodules.helpers_vz as helpers_vz
imp.reload(helpers_vz)

In [ ]:
class CurVariableHelper(helpers_vz.VariableBaseHelper):
    COL_D2RIM = 'DistRim'
    COL_GENE = 'gene'
    COL_GENE_UNTAGGED = 'gene_untagged'
    COL_TAG = 'tag'
    COL_DOXO = 'doxocycline'
    COL_DILUTION = 'dilution'
    COL_GOODNAME = 'goodname'
    COL_ISNB = 'isnb'
    COL_FITTED = 'fitted'
    COL_RESID = 'residual'
    COL_IMGID = db.images.image_id.key
    COL_OBJ_NR = db.objects.object_number.key
    COL_FC = 'fc'
    COL_P = 'p'
    COL_DF = 'DF'
    COL_DELTA = 'delta'
    COL_TSTAT = 't'
    COL_TAGSTAT = 'TagStat'
    COL_POSSTAT = 'PosStat'
    COL_WORKING = 'working'
    COL_N = 'n'
    COL_N_OVEREXPR = 'n_overexpr'
    COL_FC_CENS = 'fc_cens'
    COL_NB = 'nb'
    COL_P_CORR = 'p_corrected'
    COL_ISSIG = 'is_sig_sel'
    COL_FITCONDITIONNAME = 'FitConditionName'
    COL_VALUES = db.object_measurements.value.key

    COL_COEFNAME = 'coefname'
    
    LAB_CELLLINE = 'Cellline'
    LAB_TP = 'Timepoint'
    LAB_CONC = 'Size'
    
V = CurVariableHelper

In [ ]:
V.COL_MODTYPE = 'modtype'
V.COL_MODEL = 'model'
V.COL_MODELCLASS = 'modelclass'
V.COL_METAL = 'metal'
V.COL_MARKER_CLASS = 'marker_class'
V.COL_IS_CC = 'is_cc'

V.COL_CONDITION = V.COL_CONDNAME
V.COL_TP = 'timepoint'
V.COL_CONC = 'concentration'
V.COL_CELLLINE = 'cellline'

V.COL_CHANNEL = 'channel'

In [ ]:
class C:
    main_conds = [f'{cl}_c{c}_te%_tp{tp}' 
                  for cl, c, tp in 
                  [('293T', '0.5', '96'),
                   ('HT29','0.5','96'),
                   ('DLD1','0.5', '96'),
                   ('T47D', '1.0', '96')] ]
    main_conds_cl = ['293T', 'HT29', 'DLD1', 'T47D']
    all_conds =  [f'{cl}_c{c}_te%_tp{tp}'  for cl in ['293T', 'HT29', 'DLD1', 'T47D'] for tp in ['72', '96'] for c in [  '0.25', '0.5', '1.0'] ]
    fig2_cond = [main_conds[1]]

In [ ]:
V.COL_MEASTYPE = db.measurement_types.measurement_type.key
V.COL_MEASID = db.measurements.measurement_id.key
V.COL_VALUE = db.object_measurements.value.key
MEAS_INTENSITY = 'Intensity'

In [ ]:
cmap_cells = [colors.to_hex(c) for c in sns.color_palette('deep')]

In [ ]:
# I added consoring as there were hugh outliers in the data
def censor_dat(x, q=99.9):
    x = np.copy(x)
    p = np.percentile(x,q=q)
    x[x> p] = p
    return x

def cur_logtransf(x):
    return np.log10(x+0.1)

def cur_transf(x):
    x= censor_dat(x)
    x= cur_logtransf(x)
    return x

def transf_intensities(dat, dat_measmeta):
    ids = dat_measmeta.loc[dat_measmeta[V.COL_MEASTYPE] == MEAS_INTENSITY, V.COL_MEASID]
    fil = dat[V.COL_MEASID].isin(ids)
    dat = dat.copy()
    dat.loc[fil, V.COL_VALUE] = dat.loc[fil, :].groupby(V.COL_MEASID).transform(cur_transf)
    return dat

def get_imgs_for_cond(condition_name):
    q = (bro.session.query(db.images.image_id)
     .join(db.conditions)
     .join(db.valid_images)
     .filter(db.conditions.condition_name.like(f'{condition_name}%')))
    return [i[0] for i in q.all()] 

def get_condids_for_cond(condition_name):
    q = (bro.session.query(db.conditions.condition_id)
     .join(db.images)
     .join(db.valid_images)
     .filter(db.conditions.condition_name.like(f'{condition_name}%')))
    return [i[0] for i in q.all()] 

def get_valid_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [ ]:
fn_config = sm.input.fn_config
fn_modstats_out = sm.input.fn_config
#fn_modstats_perrep ='../../figures/marker_relations_r2_fullqc_perrep.csv'
fol_out_paper = pathlib.Path(sm.output.fol_out)
fn_params = sm.input.fn_modstats_params
fol_out = pathlib.Path(sm.output.fol_out)
os.makedirs(fol_out, exist_ok=True)
os.makedirs(fol_out_paper, exist_ok=True)

In [ ]:




bro = spb.get_bro(fn_config)


hpr = helpers_vz.HelperVZ(bro)

dat_pannelcsv = hpr.get_pannelcsv()


dat_pannelcsv.loc[dat_pannelcsv['metal'] == 'Gd155',V.COL_WORKING] = True

dat_measmeta = hpr.get_measuremeta(dat_pannelcsv,additional_measfilt=sa.and_(db.stacks.stack_name == 'ObjectStack',
                                                                            db.measurements.measurement_name.in_(['dist-rim','NbMeandist-rim']),
                                                                            db.ref_planes.channel_name == 'object'))

dat_imgmeta = hpr.get_imgmeta()
#dat_imgmeta[V.COL_CONDLEVEL] = dat_imgmeta[V.COL_CONDID].map(str)
#dat_imgmeta[V.COL_SITELEVEL] = dat_imgmeta[V.COL_SITEID].map(str)


fil_good_meas = hpr.get_fil_good_meas(dat_measmeta)







#dat_measmeta = dat_measmeta.merge(dat_pannelcsv[[V.COL_METAL, V.COL_MARKER_CLASS]],how='left')
dat_measmeta = dat_measmeta.merge(dat_pannelcsv, how='left')
dat_measmeta[V.COL_IS_CC] = dat_measmeta[V.COL_IS_CC] == 1
#dat_measmeta[V.COL_IS_CC] = False
dat_measmeta = dat_measmeta.set_index(V.COL_MEASID, drop=False)

In [ ]:
chan_dict = {r['metal']: r[V.COL_GOODNAME] for _, r in bro.data.pannel.iterrows()}

In [ ]:
import re

In [ ]:
V.COL_PARAMS = 'params' 
V.COL_VAR = 'variable'
MOD_DIST = 'dist'

In [ ]:
dat_mod_params = pd.read_csv(fn_params)

In [ ]:
MOD_DIST = 'site'

In [ ]:
V.COL_PARAMS_SITE = V.COL_PARAMS+V.COL_SITELEVEL
V.COL_PARAMS_IMG = V.COL_PARAMS+V.COL_IMGLEVEL
V.COL_PARAMS_COND = V.COL_PARAMS+V.COL_CONDLEVEL
V.COL_FITCOND_NAME = V.COL_CONDNAME+'Fit'
def get_cor_params(dat_mod_params):
    fil =  (dat_mod_params[V.COL_MODELCLASS] == MOD_DIST)
    cur_params = dat_mod_params.loc[fil, :]
    fil = cur_params[V.COL_VAR].map(lambda x: ((x.startswith(V.COL_CONDLEVEL))))
    cur_params_cond = cur_params.loc[fil, :].copy()
    fil = cur_params[V.COL_VAR].map(lambda x: ((x.startswith(V.COL_IMGLEVEL))))
    cur_params_img = cur_params.loc[fil, :].copy()
    cur_params_img[V.COL_IMGLEVEL] = cur_params_img.groupby([V.COL_VAR])[V.COL_VAR].transform(
        lambda x: x.values[0].replace(V.COL_IMGLEVEL,'').replace('[T.','').replace(']',''))
    cur_params_img = cur_params_img.rename(columns={V.COL_PARAMS: V.COL_PARAMS_IMG,
                                                     V.COL_CONDNAME: V.COL_FITCOND_NAME})
    #cur_params_site = cur_params_site.rename(columns={V.COL_PARAMS: V.COL_PARAMS_SITE,
    #                                                 V.COL_CONDNAME: V.COL_FITCOND_NAME})
    #cur_params_cond[V.COL_CONDLEVEL] = cur_params_cond.groupby([V.COL_VAR])[V.COL_VAR].transform(
    #    lambda x: x.values[0].replace(V.COL_CONDLEVEL,'').replace('[T.','').replace(']',''))
    #cur_params_cond = cur_params_cond.rename(columns={V.COL_PARAMS: V.COL_PARAMS_COND,
    #                                                 V.COL_CONDNAME: V.COL_FITCOND_NAME})
    
    return cur_params_img

def cor_intensities(dat,  dat_sitecor, dat_measmeta, dat_imgmeta, condition):
    ids = dat_measmeta.loc[dat_measmeta[V.COL_MEASTYPE] == MEAS_INTENSITY, V.COL_MEASID]
    fil = dat.index[dat[V.COL_MEASID].isin(ids)]
    dat = dat.copy()
    dat_cor = (dat[[V.COL_MEASID, V.COL_IMID, V.COL_OBJID]]
               .merge(dat_measmeta[[V.COL_MEASID, V.COL_CHANNELNAME]])
     .merge(dat_imgmeta[[V.COL_IMID, V.COL_SITELEVEL, V.COL_CONDITION, V.COL_CONDLEVEL, V.COL_IMGLEVEL]])
     #.merge(dat_condcor[[V.COL_CHANNELNAME, V.COL_PARAMS_COND, V.COL_CONDLEVEL]]
     .merge(dat_sitecor[[V.COL_CHANNELNAME, V.COL_PARAMS_IMG, V.COL_IMGLEVEL,V.COL_FITCOND_NAME]].query(f'{V.COL_FITCOND_NAME} == "{condition}"'))
              )
              
    dat= dat.merge(dat_cor[[V.COL_MEASID, V.COL_OBJID, V.COL_PARAMS_IMG]], how='left').fillna(0)

    dat[V.COL_VALUE] = dat[V.COL_VALUE] - dat[V.COL_PARAMS_IMG]
    dat = dat.drop([V.COL_PARAMS_IMG], axis=1)
    return dat

In [ ]:
dat_sitecor = get_cor_params(dat_mod_params)

In [ ]:
%%time
dat_hm_all = hpr.get_data( fil_good_meas=fil_good_meas, object_type='cell')



In [ ]:
dat_d2rim = hpr.get_d2rim()

Calculate number of cells per condition

In [ ]:
dat_imgmeta[V.COL_CONDNAME] = dat_imgmeta[db.images.condition_id.key].replace({i: c for c in C.all_conds for i in get_condids_for_cond(c)})

In [ ]:
def get_hm_data(cur_cond):
    img_ids=get_imgs_for_cond(cur_cond)
    dat = dat_hm_all.query(f'{V.COL_IMID} in {img_ids}')
    dat = transf_intensities(dat, dat_measmeta)
    #dat_transf_cor = cor_intensities(dat_transf, dat_sitecor, dat_condcor, dat_measmeta, dat_imgmeta)
    return dat

In [ ]:
conds = C.main_conds

In [ ]:
dat_measmeta = dat_measmeta.reset_index(drop=True)

In [ ]:
%%time
c = conds[1]
d = get_hm_data(c)
d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, c)

In [ ]:
dat_cms = [get_hm_data(c) for c in conds]

In [ ]:
V.COL_ISNB = 'isnb'
def plt_clustmat(dat_meas_raw, dat_measmeta, title=None, link=None,
                         linkage_method='average', linkage_metric='Euclidean',
                         corrmethod='pearson'):       
    dat_meas = dat_meas_raw.pivot_table(values=V.COL_VALUES,
                        index=V.COL_OBJID,
                        columns=V.COL_MEASID)

    dat_meas = dat_meas.dropna()
    if corrmethod == 'pearson':
        corrdat = np.corrcoef(dat_meas.T)
    elif corrmethod == 'spearman':
        corrdat, p = stats.spearmanr(dat_meas)
    else:
        raise f'{corrmethod} is not a valid corrmethod'
        
    dat_measmeta = dat_measmeta.set_index(V.COL_MEASID)

    colnames = dat_measmeta.loc[dat_meas.columns.values,:].apply(
        lambda x: ' - '.join([x[V.COL_ISNB], x[V.COL_GOODNAME]]), axis=1)
    
    c = sns.color_palette("Set2", 10)

    cols = []

    for ch in colnames:
        if ch.startswith('Int - object'):
            cols.append(c[5])
        elif ch.startswith('Int -'):
            cols.append(c[7])
        elif ch.startswith('Nb -'):
            cols.append(c[2])
        else:
            raise('Invalid name')

    cmap =sns.color_palette("RdBu_r", 21)
    hide_link = link is not None
    if link is None:
        link = hierarchy.linkage(corrdat, method=linkage_method,
                                 metric=linkage_metric, optimal_ordering=True)
    cg = sns.clustermap(pd.DataFrame(corrdat, index=colnames, columns=colnames),
                       row_colors=cols, figsize=(15,15), row_linkage=link, col_linkage=link,
                       center=0,cmap=cmap,  yticklabels=1, xticklabels=1,
                       dendrogram_ratio=0.1)
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
    plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), rotation=-90)
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), size=8)
    plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), size=8)
    #cg.fig.subplots_adjust(bottom=0.3)
    #cg.fig.subplots_adjust(right=0.7)
    #sns.set(font_scale=0.2)
    if hide_link:
        cg.ax_row_dendrogram.set_visible(False)
        cg.ax_col_dendrogram.set_visible(False)
    if title is not None:
        plt.suptitle(title)
    return cg

In [ ]:
link_metric = 'correlation'
link_method = 'average'
cgs_cor = [plt_clustmat(cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, c),
                                 dat_measmeta, title=c+'-Corr',
                                linkage_metric=link_metric,
                                 linkage_method=link_method,
                                ) for d,c in zip([dat_cms[1]], [conds[1]])]

for cg, c in zip(cgs_cor, conds):
    cg.savefig(fol_out_paper / get_valid_filename(f'corrmat_{c}_{link_metric}_{link_method}.pdf'))

In [ ]:
link_metric = 'correlation'
link_method = 'complete'
cgs_cor = [plt_clustmat(cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, c),
                                 dat_measmeta, title=c+'-Corr',
                             corrmethod='spearman',
                                linkage_metric=link_metric,
                                
                                  linkage_method=link_method,
                                ) for d,c in zip([dat_cms[1]], [conds[1]])]

In [ ]:
cgs_cor = [plt_clustmat(cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, c),
                                 dat_measmeta, title=c+'-Corr',
                                linkage_metric='euclidean',
                                 linkage_method='ward'
                                ) for d,c in zip([dat_cms[1]], [conds[1]])]

In [ ]:
link_metric = 'correlation'
link_method = 'complete'
cgs_cor = [plt_clustmat(cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, c),
                                 dat_measmeta, title=c+'-Corr',
                                linkage_metric=link_metric,
                                 linkage_method=link_method,
                                ) for d,c in zip(dat_cms, conds)]

for cg, c in zip(cgs_cor, conds):
    cg.savefig(fol_out_paper / get_valid_filename(f'corrmat_{c}_{link_metric}_{link_method}.pdf'))

In [ ]:
cgs_cor[1]

In [ ]:
cgs_cor = [plt_clustmat(cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, c),
                                 dat_measmeta, title=c+'-Corr',
                                linkage_metric='cosine',
                                 linkage_method='complete'
                                ) for d,c in zip(dat_cms, conds)]

In [ ]:
cgs_corcor = [plt_clustmat(cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, c),
                                 dat_measmeta, title=c+'-Corr',
                                linkage_metric='euclidean',
                                 linkage_method='average'
                                ) for d,c in zip(dat_cms, conds)]

Quick check if the correlations make sense

In [ ]:
dat = get_hm_data(C.fig2_cond[0])

In [ ]:
dcorr = cor_intensities(dat, dat_sitecor, dat_measmeta, dat_imgmeta, C.fig2_cond[0])

In [ ]:
meas_d2rim = dat_measmeta.query(f'{V.COL_MEASNAME} == "dist-rim"')[V.COL_MEASID].iloc[0]

meas_cyclin  =  dat_measmeta.query(f'({V.COL_MEASNAME} == "MeanIntensityComp") &\
                ({V.COL_CHANNELNAME} == "Nd145")')[V.COL_MEASID].iloc[0]
d= dcorr.query(f'{V.COL_MEASID} in {[meas_d2rim, meas_cyclin]}').pivot_table(V.COL_VALUE, V.COL_OBJID,
                                                                         V.COL_MEASID)
plt.hexbin(d[meas_d2rim], d[meas_cyclin])
plt.show()
d= dat.query(f'{V.COL_MEASID} in {[meas_d2rim, meas_cyclin]}').pivot_table(V.COL_VALUE, V.COL_OBJID,
                                                                         V.COL_MEASID)
plt.hexbin(d[meas_d2rim], d[meas_cyclin])

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from patsy.builtins import Q
from patsy import bs

In [ ]:
d= dcorr.query(f'{V.COL_MEASID} in {[meas_d2rim, meas_cyclin]}').pivot_table(V.COL_VALUE, V.COL_OBJID,
                                                                         V.COL_MEASID)
ren = helpers_vz.Renamer()
d=d.rename(columns=ren.rename)
x=ren.rename(meas_d2rim)
y=ren.rename(meas_cyclin)
mod = smf.ols(f'{y}~bs({x},df=10)',d).fit()

In [ ]:
mod.summary()

In [ ]:
plt.hexbin(d[x], d[y])
plt.scatter(d[x], mod.predict(d[x]), s=1)

In [ ]:
plt.scatter(mod.fittedvalues, mod.resid_pearson,s=1)

In [ ]:


meas_cyclin  =  dat_measmeta.query(f'({V.COL_MEASNAME} == "MeanIntensityComp") &\
                ({V.COL_CHANNELNAME} == "Yb171")')[V.COL_MEASID].iloc[0]
d= dcorr.query(f'{V.COL_MEASID} in {[meas_d2rim, meas_cyclin]}').pivot_table(V.COL_VALUE, V.COL_OBJID,
                                                                         V.COL_MEASID)
plt.hexbin(d[meas_d2rim], d[meas_cyclin])

In [ ]:
meas_cyclin  =  dat_measmeta.query(f'({V.COL_MEASNAME} == "MeanIntensityComp") &\
                ({V.COL_CHANNELNAME} == "Yb171")')[V.COL_MEASID].iloc[0]
d= dat.query(f'{V.COL_MEASID} in {[meas_d2rim, meas_cyclin]}').pivot_table(V.COL_VALUE, V.COL_OBJID,
                                                                         V.COL_MEASID)
plt.hexbin(d[meas_d2rim], d[meas_cyclin])

Make a heatmap: d2rim bin vs marker

In [ ]:
dat_d2rim.columns

In [ ]:
V.COL_D2RIM = 'distrim'
V.COL_D2RIM_BIN = V.COL_D2RIM+'bin'
V.bin_stepsize = 10

In [ ]:
dat_d2rim[V.COL_D2RIM_BIN] = np.digitize(dat_d2rim[V.COL_D2RIM], bins=np.arange(0,400,V.bin_stepsize))*V.bin_stepsize

In [ ]:
meas_d2rim = dat_measmeta.query(f'{V.COL_MEASNAME} == "dist-rim"')[V.COL_MEASID].iloc[0]



In [ ]:
def plot_d2rim_hm(dat, dat_d2rim, min_n=50, stat_agg='median', labs=[V.COL_GOODNAME],
                 meas_name='MeanIntensityComp', title='', cmax=None, figsize=None, link=None):
    cmap =sns.color_palette("RdBu_r", 101)
    dat_bin = (dat
           .merge(dat_d2rim[[V.COL_OBJID, V.COL_D2RIM_BIN]])
           .groupby([V.COL_MEASID, V.COL_D2RIM_BIN])[V.COL_VALUE]
           .agg(**{V.COL_VALUE: stat_agg, 'n': 'count'})
           #.apply(lambda x: stats.trim_mean(x,0.01))
          .reset_index(drop=False)
            .query(f'n > {min_n}')
          )
    dat_bin[V.COL_VALUE] =dat_bin.groupby(V.COL_MEASID)[V.COL_VALUE].transform(lambda x: x-x.mean())
    dat_bin =dat_bin.merge(dat_measmeta[[V.COL_MEASID, V.COL_MEASNAME]+labs])
    if meas_name is not None:
        dat_bin = dat_bin.query(f'{V.COL_MEASNAME} == "{meas_name}"')
    dat_bin = (dat_bin
               .pivot_table(index=labs,columns=V.COL_D2RIM_BIN, values=V.COL_VALUE))
    if link is None:
        link = hierarchy.linkage(dat_bin, metric='correlation', method='complete', optimal_ordering=True)
    cg=sns.clustermap(dat_bin,
               col_cluster=False, row_linkage=link,yticklabels=1, xticklabels=1,
                      cmap=cmap, center=0,vmin=-cmax, vmax=cmax, figsize=figsize,
                      dendrogram_ratio=0.1
        
              )
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
    plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), rotation=-90)
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), size=8)
    plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), size=8)
    #cg.fig.subplots_adjust(bottom=0.3)
    #cg.fig.subplots_adjust(right=0.7)
    plt.title(title)
    return cg




In [ ]:
cgs =[plot_d2rim_hm(cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, c), dat_d2rim, title=c+'-Corr',
                   stat_agg=lambda x: stats.trim_mean(x, 0.01), cmax=np.log10(3),figsize=(4,6)) for d,c in zip(dat_cms, conds)]

In [ ]:
for cg, c in zip(cgs, conds):
    cg.savefig(fol_out_paper / get_valid_filename(f'main_dist_vs_mark_{c}.pdf'))

In [ ]:
plt.close('all')

In [ ]:
def plot_d2rim_hm_cont(dat, dat_d2rim, min_n=50, stat_agg='median', labs=[V.COL_GOODNAME, V.COL_CHANNELNAME],
                 meas_name='MeanIntensityComp', title='', cmax=None):
    cmap =sns.color_palette("viridis", 101)
    dat_bin = (dat
           .merge(dat_d2rim[[V.COL_OBJID, V.COL_D2RIM_BIN]])
           .groupby([V.COL_MEASID, V.COL_D2RIM_BIN])[V.COL_VALUE]
           .agg(**{V.COL_VALUE: stat_agg, 'n': 'count'})
           #.apply(lambda x: stats.trim_mean(x,0.0|1))
          .reset_index(drop=False)
            .query(f'n > {min_n}')
          )
    dat_bin[V.COL_VALUE] =dat_bin.groupby(V.COL_MEASID)[V.COL_VALUE].transform(lambda x: x-x.min())
    dat_bin =dat_bin.merge(dat_measmeta[[V.COL_MEASID, V.COL_MEASNAME]+labs])
    if meas_name is not None:
        dat_bin = dat_bin.query(f'{V.COL_MEASNAME} == "{meas_name}"')
    dat_bin = (dat_bin
               .pivot_table(index=labs,columns=V.COL_D2RIM_BIN, values=V.COL_VALUE))
    link = hierarchy.linkage(dat_bin, metric='correlation', method='complete', optimal_ordering=True)
    cg=sns.clustermap(dat_bin,
               col_cluster=False, row_linkage=link,yticklabels=1, xticklabels=1,
                      cmap=cmap
                      #, center=0,vmin=-cmax, vmax=cmax

              )
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
    plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), rotation=-90)
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), size=8)
    plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), size=8)
    cg.fig.subplots_adjust(bottom=0.3)
    cg.fig.subplots_adjust(right=0.7)
    plt.title(title)
    return cg

In [ ]:
cgs =[plot_d2rim_hm_cont(cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, c), dat_d2rim, title=c+'-Corr',
                   stat_agg=lambda x: stats.trim_mean(x, 0.01), cmax=np.log10(3)) for d,c in zip(dat_cms, conds)]

Plot distance to border vs pS6 in 293T, c1, tp=96

In [ ]:
cond = '293T_c1.0_te%_tp96'
dat = get_hm_data(cond)

dcorr = cor_intensities(dat, dat_sitecor, dat_measmeta, dat_imgmeta, C.fig2_cond[0])

In [ ]:
meas_d2rim = dat_measmeta.query(f'{V.COL_MEASNAME} == "dist-rim"')[V.COL_MEASID].iloc[0]

meas_cyclin  =  dat_measmeta.query(f'({V.COL_MEASNAME} == "MeanIntensityComp") &\
                ({V.COL_CHANNELNAME} == "Yb171")')[V.COL_MEASID].iloc[0]
d= dcorr.query(f'{V.COL_MEASID} in {[meas_d2rim, meas_cyclin]}').pivot_table(V.COL_VALUE, V.COL_OBJID,
                                                                         V.COL_MEASID)
plt.hexbin(d[meas_d2rim], d[meas_cyclin])
plt.show()
d= dat.query(f'{V.COL_MEASID} in {[meas_d2rim, meas_cyclin]}').pivot_table(V.COL_VALUE, V.COL_OBJID,
                                                                         V.COL_MEASID)
plt.hexbin(d[meas_d2rim], d[meas_cyclin])

Calculate std

In [ ]:
def get_d2rim_hmdat(dat, dat_d2rim, stat_agg=np.mean, cmax=np.log10(3),min_n=100):
    dat_bin = (dat
           .merge(dat_d2rim[[V.COL_OBJID, V.COL_D2RIM_BIN]])
           .groupby([V.COL_MEASID, V.COL_D2RIM_BIN])[V.COL_VALUE]
           .agg(**{V.COL_VALUE: stat_agg, 'n': 'count'})
           #.apply(lambda x: stats.trim_mean(x,0.01))
          .reset_index(drop=False)
            .query(f'n > {min_n}')
          )
    
    dat_bin[V.COL_VALUE] =dat_bin.groupby(V.COL_MEASID)[V.COL_VALUE].transform(lambda x: x-x.mean())     
    dat_bin = (dat_bin
               .pivot_table(index=[V.COL_MEASID],columns=V.COL_D2RIM_BIN, values=V.COL_VALUE))
    return dat_bin
    


def plot_d2rim_hm(dat, dat_d2rim, min_n=50, stat_agg='median', labs=[V.COL_GOODNAME],
                 meas_name='MeanIntensityComp', title='', cmax=None, figsize=None, link=None):
    cmap =sns.color_palette("RdBu_r", 101)
    dat_bin = (dat
           .merge(dat_d2rim[[V.COL_OBJID, V.COL_D2RIM_BIN]])
           .groupby([V.COL_MEASID, V.COL_D2RIM_BIN])[V.COL_VALUE]
           .agg(**{V.COL_VALUE: stat_agg, 'n': 'count'})
           #.apply(lambda x: stats.trim_mean(x,0.01))
          .reset_index(drop=False)
            .query(f'n > {min_n}')
          )
    

    dat_bin[V.COL_VALUE] =dat_bin.groupby(V.COL_MEASID)[V.COL_VALUE].transform(lambda x: x-x.mean())
    dat_bin =dat_bin.merge(dat_measmeta[[V.COL_MEASID, V.COL_MEASNAME]+labs])
    if meas_name is not None:
        dat_bin = dat_bin.query(f'{V.COL_MEASNAME} == "{meas_name}"')
        

    dat_bin = (dat_bin
               .pivot_table(index=[V.COL_MEASID]+labs,columns=V.COL_D2RIM_BIN, values=V.COL_VALUE))
    if link is None:
        link = hierarchy.linkage(dat_bin, metric='correlation', method='complete', optimal_ordering=True)
    
    
    cg=sns.clustermap(dat_bin,
               col_cluster=False, row_linkage=link,yticklabels=1, xticklabels=1,
                      cmap=cmap, center=0,vmin=-cmax, vmax=cmax, figsize=figsize,
                      dendrogram_ratio=0.1
        
              )
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
    plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), rotation=-90)
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), size=8)
    plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), size=8)
    #cg.fig.subplots_adjust(bottom=0.3)
    #cg.fig.subplots_adjust(right=0.7)
    plt.title(title)
    return cg

In [ ]:
def calc_cormat(dat, corrmethod='pearson'):       
    dat_meas = dat.pivot_table(values=V.COL_VALUES,
                        index=V.COL_OBJID,
                        columns=V.COL_MEASID)

    dat_meas = dat_meas.dropna()
    if corrmethod == 'pearson':
        corrdat = np.corrcoef(dat_meas.T)
    elif corrmethod == 'spearman':
        corrdat, p = stats.spearmanr(dat_meas)
    else:
        raise f'{corrmethod} is not a valid corrmethod'
        
    corrdat = pd.DataFrame(corrdat, index=dat_meas.columns, columns=dat_meas.columns)
    return corrdat
    
    
def get_measids(measmeta, col, val):
    return measmeta.query(f'{col} == "{val}"')[V.COL_MEASID]
    
def calc_int_linkage(cormat, dat_measmeta, ref='Int', **kwargs):
    int_measids =  get_measids(dat_measmeta, V.COL_ISNB, ref)
    cm = cormat.loc[cormat.index.isin(int_measids), cormat.columns.isin(int_measids)]
    link_clust = hierarchy.linkage(cm, optimal_ordering=True, **kwargs)
    return link_clust, cm.columns



In [ ]:
C.fig2_cond

In [ ]:
%%time
c = C.fig2_cond[0]
d = get_hm_data(c)
d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, c)

In [ ]:
import matplotlib.colors as colors

In [ ]:
def label_measids(dat_measmeta, label, idxvar=V.COL_MEASID):
    lab_series = dat_measmeta[[idxvar, label]].drop_duplicates().set_index(idxvar)
    return lab_series[label]

def extend_linkage(link):
    """
    Adds another row at index 0
    """
    link = link.copy()
    link[:, :2] +=1
    new_row = [[0, max(link[:,0].max(),link[:,1].max())+1, link[:,2].max(), link[-1,3]+1]]
    return np.concatenate((link, new_row))

def plot_heatmaps(data, dat_measmeta, dat_d2rim, maskfkt=None,
                 corrmethod='pearson', link_metric = 'correlation',
                link_method = 'complete', dist_sep = True):
    label = V.COL_GOODNAME
    cmap =sns.color_palette("RdBu_r", 21)
    dend_space = 0.05
    
    cormat = calc_cormat(data,corrmethod=corrmethod)
    if dist_sep:
        m_dist = dat_measmeta.query(f'{V.COL_CHANNELNAME} == "object" & {V.COL_ISNB} == "Int"')[V.COL_MEASID].iloc[0]
        print(m_dist)
        link_clust, link_measids = calc_int_linkage(cormat, dat_measmeta.query(f'{V.COL_MEASID} not in [{m_dist}]'), method=link_method, metric=link_metric)
        link_clust = extend_linkage(link_clust)
        link_measids = [m_dist] + list(link_measids)
    else:
        link_clust, link_measids = calc_int_linkage(cormat, dat_measmeta, method=link_method, metric=link_metric)
    
    link_ord = hierarchy.leaves_list(link_clust)
    # gete the channel names of these meas ids
    link_channelnames = (dat_measmeta
                       .set_index(V.COL_MEASID).loc[link_measids,:]
                       [V.COL_CHANNELNAME]
                      )
    
    int_meas = get_measids(dat_measmeta, V.COL_ISNB, 'Int')
    

    fil_measids = int_meas
    cm = cormat.loc[cormat.index.isin(fil_measids), cormat.columns.isin(fil_measids)]
    lab = label_measids(dat_measmeta, V.COL_CHANNELNAME)
    cm = pd.DataFrame(cm.values, index=lab.loc[cm.index], columns=lab.loc[cm.columns])
    cm = cm.loc[link_channelnames, link_channelnames]
    
    lab = label_measids(dat_measmeta, V.COL_GOODNAME, V.COL_CHANNELNAME)
    cm = pd.DataFrame(cm.values, index=lab.loc[cm.index], columns=lab.loc[cm.columns])
    cm = cm.iloc[:,link_ord]
    if maskfkt is not None:
        mask = maskfkt(cm.values)
    else:
        mask = None
    cg_int = sns.clustermap(cm, row_linkage=link_clust, col_cluster=False, cmap=cmap, vmin=-1, vmax=1,
               yticklabels=1, xticklabels=1,label='big', mask=mask)
    
    #cg_int.ax_col_dendrogram.set_visible(False)
    cg_int.ax_heatmap.set_yticks([])
    cg_int.ax_heatmap.set_ylabel('')
    cg_int.ax_heatmap.set_xlabel('')
    
    
    nb_meas = get_measids(dat_measmeta, V.COL_ISNB, 'Nb')
    cm = cormat.loc[cormat.index.isin(int_meas), cormat.columns.isin(nb_meas)]
    lab = label_measids(dat_measmeta, V.COL_CHANNELNAME)
    cm = pd.DataFrame(cm.values, index=lab.loc[cm.index], columns=lab.loc[cm.columns])
    cm = cm.loc[link_channelnames, link_channelnames]
    
    lab = label_measids(dat_measmeta, V.COL_GOODNAME, V.COL_CHANNELNAME)
    cm = pd.DataFrame(cm.values, index=lab.loc[cm.index], columns=lab.loc[cm.columns])
    cm = cm.iloc[link_ord,link_ord]
    if maskfkt is not None:
        mask = maskfkt(cm.values)
    else:
        mask = None
    cg_nb = sns.clustermap(cm , row_cluster=False, col_cluster=False, cmap=cmap, vmin=-1, vmax=1,
               yticklabels=1, xticklabels=1,label='big', mask=mask)
    
    
    d_dat = data.query(f'{V.COL_MEASID} in {list(int_meas)}').copy()
    # log transform d2rim
    d2rim_idx = dat_measmeta.query(f'{V.COL_MEASID} in {list(int_meas)}').query(f'{V.COL_MEASNAME} == "dist-rim"')[V.COL_MEASID]
    d_dat.loc[d_dat[V.COL_MEASID].isin(d2rim_idx), V.COL_VALUE] = d_dat.loc[d_dat[V.COL_MEASID].isin(d2rim_idx), V.COL_VALUE].map(np.log10)
    
    cm = get_d2rim_hmdat(d_dat, dat_d2rim, stat_agg=np.mean,min_n=100)
                              
    lab = label_measids(dat_measmeta, V.COL_CHANNELNAME)
    cm = pd.DataFrame(cm.values, index=lab.loc[cm.index], columns=cm.columns)
    cm = cm.loc[link_channelnames, :]
    
    lab = label_measids(dat_measmeta, V.COL_GOODNAME, V.COL_CHANNELNAME)
    cm = pd.DataFrame(cm.values, index=lab.loc[cm.index], columns=cm.columns)
                         
    cmax = np.log10(3)
    cg_d2rim= sns.clustermap(cm.iloc[link_ord, :], col_cluster=False, row_cluster=False,yticklabels=1, xticklabels=2,
                      cmap=cmap, center=0,vmin=-cmax, vmax=cmax, figsize=(2,10),
                      dendrogram_ratio=0.1,label='big'
              )
                       
                       
    cgs = [cg_int, cg_nb]
    for cg in cgs:
        cg.ax_heatmap.set_yticks([])
    
    cgs += [cg_d2rim]
    
    for cg in cgs:                        
        cg.ax_heatmap.set_ylabel('')
        cg.ax_heatmap.set_xlabel('')
        cg.ax_heatmap.tick_params(labelsize=13)
        plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), rotation=-90)
        
    return cormat, cgs

In [ ]:
1

In [ ]:
for cond in C.fig2_cond:
    d = get_hm_data(cond)
    d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    cormat, cgs = plot_heatmaps(d_cor, dat_measmeta, dat_d2rim,
                               link_metric = 'cosine',
                link_method = 'average')
    for cg, lab in zip(cgs, ['int', 'nb', 'dist']):
        cg.fig.suptitle(cond)
        #cg.savefig(fol_out_paper / get_valid_filename(f'corrmat_fig2_{cond}_{lab}.pdf'))

In [ ]:
for cond in C.main_conds:
    d = get_hm_data(cond)
    d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    cormat, cgs = plot_heatmaps(d_cor, dat_measmeta, dat_d2rim,
                               link_metric = 'cosine',
                link_method = 'average')
    for cg, lab in zip(cgs, ['int', 'nb', 'dist']):
        cg.fig.suptitle(cond)
        cg.savefig(fol_out_paper / get_valid_filename(f'corrmat_fig2_{cond}_{lab}.pdf'))

In [ ]:
for cond in C.main_conds:
    d = get_hm_data(cond)
    d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    cormat, cgs = plot_heatmaps(d_cor, dat_measmeta, dat_d2rim, corrmethod='spearman',
                               link_metric = 'cosine',
                link_method = 'average')
    for cg, lab in zip(cgs, ['int', 'nb', 'dist']):
        cg.fig.suptitle(cond)
        cg.savefig(fol_out_paper / get_valid_filename(f'corrmat_spearman_fig2_{cond}_{lab}.pdf'))

In [ ]:
for cond in C.fig2_cond:
    d = get_hm_data(cond)
    d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    cormat, cgs = plot_heatmaps(d_cor, dat_measmeta, dat_d2rim, maskfkt = lambda x: np.abs(x) < 0.5,
                               link_metric = 'cosine',
                link_method = 'average')
    for cg, lab in zip(cgs, ['int', 'nb', 'dist']):
        cg.fig.suptitle(cond)

In [ ]:
dat_measmeta

In [ ]:
measids = dat_measmeta.query(f'({V.COL_CHANNELNAME}=="Lu175")')[V.COL_MEASID]

cormat.loc[measids, measids]

In [ ]:
for cond in C.fig2_cond:
    d = get_hm_data(cond)
    d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    cormat, cgs = plot_heatmaps(d_cor, dat_measmeta, dat_d2rim,
                       maskfkt = lambda x: np.abs(x) < 0.5)
    for cg, lab in zip(cgs, ['int', 'nb', 'dist']):
        cg.fig.suptitle(cond)

In [ ]:
for cond in C.main_conds:
    d = get_hm_data(cond)
    #d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    cormat, cgs = plot_heatmaps(d, dat_measmeta, dat_d2rim)
    for cg, lab in zip(cgs, ['int', 'nb', 'dist']):
        cg.fig.suptitle(cond)
        cg.savefig(fol_out_paper / get_valid_filename(f'corrmat_uncorr_fig2_{cond}_{lab}.pdf'))

In [ ]:
for cond in C.fig2_cond:
    d = get_hm_data(cond)
    d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    cormat, cgs = plot_heatmaps(d_cor, dat_measmeta, dat_d2rim, maskfkt = lambda x: (x) < 0.)
    for cg, lab in zip(cgs, ['int', 'nb', 'dist']):
        cg.fig.suptitle(cond)

In [ ]:
for cond in C.fig2_cond:
    d = get_hm_data(cond)
    d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    cormat, cgs = plot_heatmaps(d_cor, dat_measmeta, dat_d2rim, maskfkt = lambda x: np.abs(x) < 0.99)
    for cg, lab in zip(cgs, ['int', 'nb', 'dist']):
        cg.fig.suptitle(cond)

In [ ]:
for cond in C.fig2_cond:
    d = get_hm_data(cond)
    d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    cormat, cgs = plot_heatmaps(d_cor, dat_measmeta, dat_d2rim, maskfkt = lambda x: np.abs(x) < 0.4)
    for cg, lab in zip(cgs, ['int', 'nb', 'dist']):
        cg.fig.suptitle(cond)

In [ ]:
idx_cc = ['Dy163', 'Sm147', 'Er168',
          'Lu175','Yb171',  'Eu153','Sm152','Dy161','Gd158'
         ]

measid_cc = dat_measmeta.query(f'({V.COL_CHANNELNAME} in {idx_cc}) &  ({V.COL_ISNB} == "Int")')[V.COL_MEASID]

cm = get_d2rim_hmdat(d_cor.query(f'{V.COL_MEASID} in {list(measid_cc)}'), dat_d2rim,
                     stat_agg=np.mean,min_n=100)

In [ ]:
(cm.stack().rename(V.COL_VALUE).reset_index().merge(dat_measmeta) 
    .assign(**{'normval':lambda x: x.groupby(V.COL_MEASID)[V.COL_VALUE]
               .transform(lambda y: y/y.max())})
 
 >>
    gg.ggplot(gg.aes(x=V.COL_D2RIM_BIN, y='normval', color = V.COL_GOODNAME))+
 gg.geom_line()
)

In [ ]:
(cm.stack().rename(V.COL_VALUE).reset_index().merge(dat_measmeta) 
    .assign(**{'normval':lambda x: x.groupby(V.COL_MEASID)[V.COL_VALUE]
               .transform(lambda y: y/y.max())})
 
 >>
    gg.ggplot(gg.aes(x=V.COL_D2RIM_BIN, y='normval', color = V.COL_GOODNAME))+
 gg.geom_line()
)

In [ ]:
chan_cc = ['Dy163', 'Sm147', 'Er168',
          'Lu175','Yb171',  'Eu153',
          #'Sm152',
          'Gd160',
          'Dy161','Gd158'
         ]
meas_int = dat_measmeta.query(f'({V.COL_ISNB} == "Int")')[V.COL_MEASID]

cm = get_d2rim_hmdat(d_cor.query(f'{V.COL_MEASID} in {list(meas_int)}'), dat_d2rim
                     .assign(**{
                         V.COL_D2RIM_BIN:
                         lambda x: np.digitize(x[V.COL_D2RIM], bins=np.arange(0,400,5))*5}),
                     stat_agg=np.mean,min_n=50)

In [ ]:
(cm.stack().rename(V.COL_VALUE).reset_index().merge(dat_measmeta) 
  .query(f'{V.COL_D2RIM_BIN} < 100') 
    .assign(**{'normval':lambda x: x.groupby(V.COL_MEASID)[V.COL_VALUE]
               .transform(lambda y: (y-y.min())/(y.max()-y.min()))})
>>
     gg.ggplot(gg.aes(x=V.COL_D2RIM_BIN, y='normval', color = V.COL_GOODNAME))+
 gg.geom_line()
)

In [ ]:
dat_mod = (d_cor.query(f'{V.COL_MEASID} in {list(meas_int)}')
     .merge(dat_d2rim)
     .groupby(V.COL_MEASID)
        .apply(lambda d: smf.ols(f'{V.COL_VALUE}~bs({V.COL_D2RIM},df=5)',d).fit())
).rename('model')
d_vals =  pd.DataFrame({V.COL_D2RIM: np.arange(2,100,1)})

tdat = (dat_mod.groupby(V.COL_MEASID).apply(lambda x: 
                    pd.DataFrame({V.COL_D2RIM: d_vals[V.COL_D2RIM],
                                  V.COL_VALUE: x.values[0].predict(d_vals)}),
                          )
  .assign(**{'val': lambda x: x.groupby(V.COL_MEASID)[V.COL_VALUE].transform(lambda d:
                                   (d-d.min())/(d.max()-d.min()))})
 .reset_index(V.COL_MEASID)
 .merge(dat_measmeta)
       )



In [ ]:
tdat = (dat_mod.groupby(V.COL_MEASID).apply(lambda x: 
                    pd.DataFrame({V.COL_D2RIM: d_vals[V.COL_D2RIM],
                                  V.COL_VALUE: x.values[0].predict(d_vals)}),
                          )
  .assign(**{'val': lambda x: x.groupby(V.COL_MEASID)[V.COL_VALUE].transform(lambda d:
                                   (d-d.min())/(d.max()-d.min()))})
 .reset_index(V.COL_MEASID)
 .merge(dat_measmeta)
       )

In [ ]:
(tdat.query(f'{V.COL_CHANNELNAME} in {chan_cc}')

 >>
gg.ggplot(gg.aes(x=V.COL_D2RIM, y='val', color=V.COL_GOODNAME))+
 gg.geom_line()

       )

In [ ]:
tdat

In [ ]:
(tdat.query(f"{V.COL_CHANNELNAME} in {['Yb171', 'Dy163', 'Er168', 'Lu175','Dy161', 'Eu153']}")

 >>
gg.ggplot(gg.aes(x=V.COL_D2RIM, y='val', color=V.COL_GOODNAME))+
 gg.geom_line()
 + gg.xlab('Distance-to-border [um]')
 + gg.ylab('Relative mean intensity [a.u.]')

       )

In [ ]:
chan = ['Dy163','Yb171','Er168', 'Lu175','Dy161' , 'Eu153']
cm = (tdat.query(f"{V.COL_CHANNELNAME} in {chan}")
 .pivot_table(index=V.COL_CHANNELNAME, columns='distrim', values='val').loc[chan,:]
)

cg_d2rim= sns.clustermap(cm, col_cluster=False, row_cluster=False,yticklabels=1, xticklabels=2,
                  center=0.5, figsize=(5,10),    cmap =sns.color_palette("RdBu_r", 21),
                  dendrogram_ratio=0.1,label='big'
          )

In [ ]:
for cond in C.main_conds:
    chan = ['Dy163','Yb171','Er168', 'Lu175','Eu153','Dy161']
    d = get_hm_data(cond)
    d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    d2r = dat_d2rim[[V.COL_OBJID,V.COL_D2RIM]].merge(d_cor[[V.COL_OBJID]])
    d2r_min, d2r_max = d2r[V.COL_D2RIM].min(), d2r[V.COL_D2RIM].max()
    dat_mod = (d_cor.query(f'{V.COL_MEASID} in {list(meas_int)}')
         .merge(dat_d2rim)
         .groupby(V.COL_MEASID)
            .apply(lambda d: smf.ols(f'{V.COL_VALUE}~bs({V.COL_D2RIM},df=5)',d).fit())
    ).rename('model')
    d_vals =  pd.DataFrame({V.COL_D2RIM: np.arange(d2r_min,d2r_max,1)})

    tdat = (dat_mod.groupby(V.COL_MEASID).apply(lambda x: 
                        pd.DataFrame({V.COL_D2RIM: d_vals[V.COL_D2RIM],
                                      V.COL_VALUE: x.values[0].predict(d_vals)}),
                              )
      .assign(**{'val': lambda x: x.groupby(V.COL_MEASID)[V.COL_VALUE].transform(lambda d:
                                       (d-d.min())/(d.max()-d.min()))})
     .reset_index(V.COL_MEASID)
     .merge(dat_measmeta)
           )

    p = (tdat.query(f"{V.COL_CHANNELNAME} in {chan}")

     >>
    gg.ggplot(gg.aes(x=V.COL_D2RIM, y='val', color=V.COL_GOODNAME))+
     gg.geom_line()+
     gg.ggtitle(f'{cond}')
          + gg.xlab('Distance-to-border [um]')
 + gg.ylab('Relative mean intensity [a.u.]')
    +gg.theme(figure_size=(5,5))
           )
    p.draw()

    cm = (tdat.query(f"{V.COL_CHANNELNAME} in {chan}")
     .pivot_table(index=V.COL_CHANNELNAME, columns='distrim', values='val').loc[chan,:]
    )
    cm.columns = map(np.round, cm.columns)
    cg_d2rim= sns.clustermap(cm, col_cluster=False, row_cluster=False,yticklabels=1, xticklabels=10,
                      center=0.5, figsize=(5,10),    cmap =sns.color_palette("RdBu_r", 199),
                      dendrogram_ratio=0.1,label='big'
              )
    plt.suptitle(cond)
    cg_d2rim

In [ ]:
for cond in C.main_conds:
    chan = ['Yb171', 'Dy163', 'Lu175','Er168', 'Eu153', 'Dy161']
    d = get_hm_data(cond)
    d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    d2r = dat_d2rim[[V.COL_OBJID,V.COL_D2RIM]].merge(d_cor[[V.COL_OBJID]])
    d2r_min, d2r_max = d2r[V.COL_D2RIM].min(), d2r[V.COL_D2RIM].max()
    dat_mod = (d_cor.query(f'{V.COL_MEASID} in {list(meas_int)}')
         .merge(dat_d2rim)
         .groupby(V.COL_MEASID)
            .apply(lambda d: smf.ols(f'{V.COL_VALUE}~cr({V.COL_D2RIM},df=10)',d).fit())
    ).rename('model')
    d_vals =  pd.DataFrame({V.COL_D2RIM: np.arange(d2r_min,d2r_max,1)})

    tdat = (dat_mod.groupby(V.COL_MEASID).apply(lambda x: 
                        pd.DataFrame({V.COL_D2RIM: d_vals[V.COL_D2RIM],
                                      V.COL_VALUE: x.values[0].predict(d_vals)}),
                              )
      .assign(**{'val': lambda x: x.groupby(V.COL_MEASID)[V.COL_VALUE].transform(lambda d:
                                       (d-d.min())/(d.max()-d.min()))})
     .reset_index(V.COL_MEASID)
     .merge(dat_measmeta)
           )

    p = (tdat.query(f"{V.COL_CHANNELNAME} in {chan}")

     >>
    gg.ggplot(gg.aes(x=V.COL_D2RIM, y='val', color=V.COL_GOODNAME))+
     gg.geom_line()+
     gg.ggtitle(f'{cond}')
          + gg.xlab('Distance-to-border [um]')
 + gg.ylab('Relative mean intensity [a.u.]')
    +gg.theme(figure_size=(5,5))
           )
    p.draw()

    cm = (tdat.query(f"{V.COL_CHANNELNAME} in {chan}")
     .pivot_table(index=V.COL_CHANNELNAME, columns='distrim', values='val').loc[chan,:]
    )

    cg_d2rim= sns.clustermap(cm, col_cluster=False, row_cluster=False,yticklabels=1, xticklabels=20,
                      center=0.5, figsize=(5,10),    cmap =sns.color_palette("RdBu_r", 21),
                      dendrogram_ratio=0.1,label='big'
              )
    plt.suptitle(cond)
    cg_d2rim

In [ ]:
C.main_conds

In [ ]:
for cond in C.main_conds:
    chan = ['Yb171', 'Dy163', 'Lu175','Er168', 'Eu153', 'Dy161']
    d_min = 2
    d_max = 81
    d = get_hm_data(cond)
    d_cor = cor_intensities(d, dat_sitecor, dat_measmeta, dat_imgmeta, cond)
    d2r = dat_d2rim[[V.COL_OBJID,V.COL_D2RIM]].merge(d_cor[[V.COL_OBJID]])
    d2r_min, d2r_max = d2r[V.COL_D2RIM].min(), d2r[V.COL_D2RIM].max()
    dat_mod = (d_cor.query(f'{V.COL_MEASID} in {list(meas_int)}')
         .merge(dat_d2rim)
         .groupby(V.COL_MEASID)
            .apply(lambda d: smf.ols(f'{V.COL_VALUE}~cr({V.COL_D2RIM},df=10)',d).fit())
    ).rename('model')
    d_vals =  pd.DataFrame({V.COL_D2RIM: np.arange(d_min,d_max,1)})
    d_vals =  pd.DataFrame({V.COL_D2RIM: np.arange(d_min,d_max,1)})

    tdat = (dat_mod.groupby(V.COL_MEASID).apply(lambda x: 
                        pd.DataFrame({V.COL_D2RIM: d_vals[V.COL_D2RIM],
                                      V.COL_VALUE: x.values[0].predict(d_vals)}),
                              )
      .assign(**{'val': lambda x: x.groupby(V.COL_MEASID)[V.COL_VALUE].transform(lambda d: d-d.mean())})
                                     #  (d-d.min())/(d.max()-d.min()))})
     .reset_index(V.COL_MEASID)
     .merge(dat_measmeta)
           )

    cm = (tdat.query(f"{V.COL_CHANNELNAME} in {chan}")
     .pivot_table(index=V.COL_CHANNELNAME, columns='distrim', values='val').loc[chan,:]
    )
    cm.index= [chan_dict[c] for c in cm.index]

    cg_d2rim= sns.clustermap(cm, col_cluster=False, row_cluster=False,yticklabels=1, xticklabels=20,
                      center=0., figsize=(2.5,2),    cmap =sns.color_palette("RdBu_r", 21),
                      dendrogram_ratio=0.1,label='big',vmin=-np.log10(3), vmax=np.log10(3))
          
    cg_d2rim.ax_heatmap.set_xlabel('Distance to border [um]')
    plt.suptitle(cond)
    cg_d2rim.savefig(fol_out_paper / f'cellcycle_{cond}.pdf')
    cg_d2rim

In [ ]:

10**0.5

In [ ]:
np.log10(2)